# 使用tensorflow实现LeNet模型

In [1]:
import tensorflow as tf
import numpy as np

class Lenet_tensorflow:
    def __init__(self):
        self.model = self.create_model()
    
    def print_activations(self,t):
        print(t.op.name, ':', t.get_shape().as_list())

    def create_model(self):
        self.features = tf.placeholder(dtype = tf.float32,shape=(None,28,28,1),name='features')
        self.labels = tf.placeholder(dtype=tf.int32,shape=(None),name = 'labels')
        
        with tf.name_scope('conv_c1') as scope:
            #c1卷积层
            c1_kernel = tf.Variable(tf.truncated_normal([5,5,1,6],dtype=tf.float32,stddev=1e-1),name='c1_kernel')
            c1_biases = tf.Variable(tf.constant(0.0,shape=[6],dtype=tf.float32),trainable=True,name='c1_biases')

            # 输入形状为[batch,h,w,in_channel]
            # 核形状为[h,w,in_channel,out_channel]
            # strides形状[1,step,step,1]
            #注意超参数padding的使用
            conv_c1 = tf.nn.conv2d(features,filter=c1_kernel,strides=(1,1,1,1),padding='VALID')
            bias = tf.nn.bias_add(conv_c1,c1_biases)
            conv_c1_output = tf.nn.sigmoid(bias,name=scope)
            self.print_activations(conv_c1_output)
            
        with tf.name_scope('maxpool_s2') as scope:
            # s2最大池化层
            maxpool_s2 = tf.nn.max_pool(conv_c1_output,ksize=(1,2,2,1),strides=(1,2,2,1),padding='VALID')
            self.print_activations(maxpool_s2)
            
        with tf.name_scope('conv_c3') as scope:
            #c3卷积层
            c3_kernel = tf.Variable(tf.truncated_normal([5,5,6,16],dtype=tf.float32,stddev=1e-1),name='c3_kernel')
            c3_biases = tf.Variable(tf.constant(0.0,shape=[16],dtype=tf.float32),trainable=True,name='c3_biases')

            # 输入形状为[batch,h,w,in_channel]
            # 核形状为[h,w,in_channel,out_channel]
            # strides形状[1,step,step,1]
            #注意超参数padding的使用
            conv_c3 = tf.nn.conv2d(maxpool_s2,filter=c3_kernel,strides=(1,1,1,1),padding='VALID')
            bias = tf.nn.bias_add(conv_c3,c3_biases)
            conv_c3_output = tf.nn.sigmoid(bias,name=scope)
            self.print_activations(conv_c3_output)
            
        with tf.name_scope('maxpool_s4') as scope:
            # s4最大池化层
            maxpool_s4 = tf.nn.max_pool(conv_c3_output,ksize=(1,2,2,1),strides=(1,2,2,1),padding='VALID')
            self.print_activations(maxpool_s4)
            maxpool_s4_flat = tf.reshape(maxpool_s4,shape=[-1,256])
            
        with tf.name_scope('dense_c5') as scope:
            # c5全连接层
            dense_c5_output = tf.layers.dense(maxpool_s4_flat, 120, activation=tf.nn.sigmoid)
            self.print_activations(dense_c5_output)
            
        with tf.name_scope('dense_f6') as scope:
            # f6全连接层
            dense_f6_output = tf.layers.dense(dense_c5_output,84,activation=tf.nn.sigmoid)
            self.print_activations(dense_f6_output)
            
        with tf.name_scope('dense_output') as scope:
            # f6全连接层
            logits = tf.layers.dense(dense_f6_output,10)
            y_prab = tf.nn.softmax(logits)
            self.print_activations(y_prab)
            
        with tf.name_scope('train') as scope:
            self.loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=labels)
            self.train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
        
        with tf.name_scope('eval') as scope:
            correct = tf.nn.in_top_k(logits, labels, 1)
            self.accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
         
    def shuffle_batch(self,x,y,n_batches):
        
        '''生成随机批量数据集'''
        
        rdn_idx = np.random.permutation(len(x))
        epochs = len(x) // n_batches
        for batch_idx in np.array_split(rnd_idx,epochs):
            x_batch = x[batch_idx]
            y_batch = y[batch_idx]
            yield x_batch,y_batch
            
    def train(self,x,y,n_epochs,n_batches):
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)
            for epoch in range(n_epochs):
                x_batch,y_batch = self.shuffle_batch(x,y,n_batches)
                feed_dict = {self.features:x_batch,self.labels:y_batch}
                _,loss_val = sess.run([self.train_op,self.loss],feed_dict=feed_dict)
                print('epoch:{},loss:{}'.format(epoch,loss_val))

C:\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
lenet_t = Lenet_tensorflow()
lenet_t.create_model()

Instructions for updating:
Colocations handled automatically by placer.


NameError: name 'features' is not defined